# Analysis of BIODYN-100 BOM

In [5]:
# Read Tidy BOM
import csv

bom_file_name = "BIODYN-100 v58 BOM Tidy.csv"

colnames = []
rows = []

with open(bom_file_name, 'r') as file:
	reader = csv.DictReader(file, delimiter=';')
	colnames = reader.fieldnames
	for row in reader:
		rows.append(row)

colnamesstr = ", ".join(colnames)
print(f"Columns: {colnamesstr}")

Columns: Part, Value, Device, Footprint Name, Detailed Description, Link, Manuf Part Number, Datasheet URL, Description, Manufacturer, DigiKey Part Number, Pricing, Raw Response
